In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import random
import os

from supervised.automl import AutoML

# 한글 폰트 설정
import platform
plt.rc('font', family='Malgun Gothic')
# 마이너스 폰트 설정
plt.rc('axes', unicode_minus=False) 
# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

""" # Warning 제거
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) 
warnings.simplefilter(action='ignore', category=UserWarning)  
pd.set_option('mode.chained_assignment',  None)   
#pd.set_option('mode.chained_assignment', 'warn') """

c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


" # Warning 제거\nimport warnings\nwarnings.simplefilter(action='ignore', category=FutureWarning) \nwarnings.simplefilter(action='ignore', category=UserWarning)  \npd.set_option('mode.chained_assignment',  None)   \n#pd.set_option('mode.chained_assignment', 'warn') "

In [2]:
train = pd.read_csv(r'data\train.csv')
test = pd.read_csv(r'data\test.csv')
target = pd.read_csv(r'data\sample_submission.csv')

In [3]:
train_x = train.drop(['ID','monthlyRent(us_dollar)'],axis=1)
train_y = train['monthlyRent(us_dollar)']

In [4]:
test_x = test.drop('ID',axis=1)

In [5]:
train_x.propertyType.unique()

array(['Apartment', 'Independent Floor', 'Independent House', 'Villa'],
      dtype=object)

In [ ]:
col = list(train.columns[6:-1])
col.append('bedrooms')
propertyType = list(train.propertyType.unique())


for i in col:
    for j in col:
#        for k in propertyType:
        if i==j:
            continue
        else:
            sns.scatterplot(data=train,x=i,y=j,hue='propertyType')
            plt.title(f'{i}-{j}')
            plt.show()



In [7]:
df=pd.DataFrame()

for i in propertyType:
    x = train_x[train_x['propertyType'] == i].describe().reset_index()
    x1=[]
    for j in list(x.index):
        x1.append(f'{i}')
    x['propertyType'] = x1
    df = pd.concat([df,x])
df = df[df['index'] !='count']

In [8]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [9]:
automl = AutoML(mode="Compete", eval_metric='mae')

In [ ]:
automl.fit(train_x, train_y)

In [ ]:
pred = automl.predict(test_x)

In [12]:
target.iloc[:, 1] = pred
target

,ID,monthlyRent(us_dollar)
0,Test_0000,185.403940
1,Test_0001,256.426172
2,Test_0002,326.162444
3,Test_0003,174.640697
4,Test_0004,79.094981
...,...,...
8688,Test_8688,359.686372
8689,Test_8689,348.863662
8690,Test_8690,206.701396
8691,Test_8691,158.459245


In [13]:
target.to_csv("target.csv", index=False)